In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

/home/vipuser/anaconda3/envs/GLM4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '/home/LLM_para/para_Qwen'
lora_path = './final_checkpoint_Qwen2_SFT_myideaTV1'
# val_dataset_path = "/home/code/chat_SQL/Dataset/GLM4/ENG_ori/My_idea_dataset/T2Q_GLM4_SFT_val_table.jsonl"

In [3]:
# model = AutoPeftModelForCausalLM.from_pretrained(
#         lora_path, attn_implementation="flash_attention_2",torch_dtype=torch.bfloat16,trust_remote_code= True, device_map='auto')
model = AutoPeftModelForCausalLM.from_pretrained(
        lora_path,torch_dtype=torch.bfloat16,trust_remote_code= True, device_map='auto')

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [01:11<00:00, 17.81s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import re
max_token = 1100
#  保证特殊字符能被识别
accept_toekn = max_token - 170
#获取REF表数据
tab_str = ""
schema_slicing = []
df = pd.read_csv("./mydataset_new/table_schema_Reference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"

    
print(len(schema_slicing))

# print(ReF_group[5])
# print("#############################")
# print(ReF_group[6])


#获取noREF表数据
df = pd.read_csv("./mydataset_new/table_schema_noReference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
#         print("+++++++++++")
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"
        
        
if len(tab_str) > 0:
    schema_slicing.append(tab_str)

print(len(schema_slicing))
# print(schema_slicing[6])
# print("#############################")
# print(schema_slicing[7])
# print(schema_slicing[8])

create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
for piece in schema_slicing:
    create_table_statements = create_table_pattern.findall(piece)
    temp_list = []
    for table in create_table_statements:
        table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
        temp_list.append(table_name.lower())
        test_total.add(table_name.lower())
        count+=1
    print(temp_list)
    table_name_group.append(temp_list)
# print(count)
print(len(test_total))
#     table_name_group.append()

100%|████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 834.41it/s]


6


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 798.36it/s]

8
['manufacturers', 'products', 'student', 'plays_games', 'sportsinfo', 'actor', 'entrepreneur', 'people', 'ref_hotel_star_ratings', 'ref_attraction_types', 'hotels', 'tourist_attractions', 'street_markets', 'shops', 'museums', 'royal_family', 'theme_parks', 'visits', 'photos', 'staff']
['tourist_attraction_features', 'wrestler', 'elimination', 'business', 'category', 'checkin', 'neighbourhood', 'review', 'tip', 'ref_detention_type', 'ref_incident_type', 'addresses', 'students', 'teachers', 'assessment_notes', 'behavior_incident', 'detention']
['student_addresses', 'students_in_detention', 'film', 'film_market_estimation', 'catalogs', 'catalog_structure', 'catalog_contents', 'catalog_contents_additional_attributes', 'routes', 'airports', 'stadium', 'game', 'injury_accident', 'physician', 'department', 'affiliated_with', 'trained_in', 'patient', 'nurse']
['appointment', 'prescribes', 'block', 'room', 'on_call', 'stay', 'undergoes', 'buildings', 'office_locations', 'region', 'party', 'me

In [6]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id 
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     print(response)
     
    return response

# instruction = row['instruction']
# input_value = row['input']
def get_instruction(schema):
    instruction = f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema}"
    return instruction

def get_input_value(question, selected_table):
    input_value = f"{question}######selected_table: {selected_table}."
    return input_value

In [7]:
df = pd.read_csv("./mydataset_new/validation_dataset_formatted_cropped.csv",encoding="utf-8")
outer_index = 0
import re
pattern = re.compile(r'-- Tables: ([\w, ]+);')
results = []
#  schema_slicing

for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row['question']
    ref_tables = row['correct_tables']
    selected_table_set = set()
    result_count = 0
    for schema in schema_slicing:
        result_count += 1
        if len(selected_table_set) > 0:
            selected_table =  ', '.join(list(selected_table_set))
        else:
            selected_table = "None"
        instruction = get_instruction(schema)
        input_value = get_input_value(question,selected_table)
        messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}]
        response = predict(messages, model, tokenizer)
        if "#None#" not in response:
            match = re.search(pattern, response)
            if match:
            # 提取匹配的表名字符串
                table_names_str = match.group(1)

                # 使用中文逗号分隔表名，这里我们替换中文逗号为英文逗号并去除空格
        #         table_names = [name.strip() for name in table_names_str.replace('，', ',').split(',')]
                table_names = table_names_str.split(', ')
                # 将table_names整合成一个由英文逗号分隔的字符串
#                 predict_table = ', '.join(table_names)
#                 
#                 print("预测表名：", predict_table)
#                 print("############################################")
                for predict_table in table_names:
                    selected_table_set.add(predict_table)
#                 if result_count == 7 or result_count == 8:
#                     print("^^^^^^^^^^^^^^^^^^^^^final_select_:",predict_table)
            else:
                predict_table = "None"
#                 selected_table_list.append(predict_table)
    if len(selected_table_set) > 0:
        total_selected_table =  ', '.join(list(selected_table_set))
    else:
        total_selected_table = "None"
    print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
    print("正确表名：",ref_tables)
    print("预测表名：",total_selected_table)
    print("############################################")
    results.append([ref_tables,total_selected_table])
print(len(predict_table))

  1%|▍                                                                                  | 1/178 [00:06<19:24,  6.58s/it]

++++++++++++++++index:0++++++++++++++++++++++++
正确表名： products, manufacturers
预测表名： products, manufacturers
############################################


  1%|▉                                                                                  | 2/178 [00:11<16:11,  5.52s/it]

++++++++++++++++index:1++++++++++++++++++++++++
正确表名： products
预测表名： products
############################################


  2%|█▍                                                                                 | 3/178 [00:16<15:05,  5.17s/it]

...

++++++++++++++++index:175++++++++++++++++++++++++
正确表名： railway
预测表名： railway
############################################


 99%|████████████████████████████████████████████████████████████████████████████████▌| 177/178 [14:02<00:04,  4.64s/it]

++++++++++++++++index:176++++++++++++++++++++++++
正确表名： book
预测表名： book
############################################


100%|█████████████████████████████████████████████████████████████████████████████████| 178/178 [14:07<00:00,  4.76s/it]

++++++++++++++++index:177++++++++++++++++++++++++
正确表名： book, publication
预测表名： publication, book
############################################
11


In [8]:
new_df = pd.DataFrame(results, columns = ['reference_tables','predicted_tables'])

In [9]:
print(new_df)

            reference_tables         predicted_tables
0    products, manufacturers  products, manufacturers
1                   products                 products
2              manufacturers            manufacturers
3              manufacturers                     None
4              manufacturers            manufacturers
..                       ...                      ...
173                     mill                     mill
174                  railway                  railway
175                  railway                  railway
176                     book                     book
177        book, publication        publication, book

[178 rows x 2 columns]


In [10]:
total_samples = len(new_df)
total_accuracy = 0
filtered_accuracy = 0
total_precision = 0
total_recall = 0
count = 0
for index, row in new_df.iterrows():
#     count+=1
#     print("index:",count)
    if not row['predicted_tables'] or pd.isna(row['predicted_tables']):
        continue
    predicted_tables = row['predicted_tables'].split(",")
    reference_tables = row['reference_tables'].split(",")
    
    # Convert to lowercase and strip whitespace for comparison
    predicted_tables = [x.lower().replace("--","").replace("**","").strip() for x in predicted_tables]
    reference_tables = [x.lower().strip() for x in reference_tables]
    
    # Calculate accuracy
    if set(predicted_tables) == set(reference_tables):
        total_accuracy += 1

    # Calculate precision and recall
    true_positives = len(set(predicted_tables) & set(reference_tables))
    false_positives = len(set(predicted_tables) - set(reference_tables))
    false_negatives = len(set(reference_tables) - set(predicted_tables))

    if true_positives == len(reference_tables):
        filtered_accuracy += 1
    
    if len(predicted_tables) > 0:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
    
    total_precision += precision
    total_recall += recall

# Calculate average precision and recall
avg_precision = total_precision / total_samples
avg_recall = total_recall / total_samples

# Calculate total accuracya
accuracy = total_accuracy / total_samples
filtered_accuracy = filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.7696629213483146
Filtered Accuracy: 0.7752808988764045
Average Precision: 0.9279026217228464
Average Recall: 0.8642322097378278


In [11]:
new_df.to_csv('pre_table.csv',index=False,encoding='utf-8')

In [12]:
"#Noooooooooooooooooo#

SyntaxError: unterminated string literal (detected at line 1) (2312456305.py, line 1)

In [ ]:
test_df = pd.read_json(val_dataset_path, lines=True)
print(test_df)

In [6]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    print(response)
     
    return response

In [7]:
# import tqdm
# df = pd.read_csv("T2Q_table_Dataset/fix_validation_dataset_formatted.csv")
# results = []
# print(df["db_id"])
# for index in range(len(df["db_id"])):
#     question = df['question'][index]
#     query = df['query'][index]
#     database_schema = df['database_schema'][index]
#     db_id = df['db_id'][index]
#     try:
#         ref_rables = ",".join(Parser(query).tables)
#     except Exception:
#         continue
#     print("\n")
#     print(response)
#     print(ref_rables)
#     print("============================")
#     results.append([ref_rables, query,question,db_id])
# new_df = pd.DataFrame(results, columns = ['reference_tables','query','question','db_id'])

In [8]:
import re
pattern = re.compile(r'-- Tables: ([\w, ]+);')
predict_list = []
index = 0
for index, row in test_df.iterrows():
    index+=1
    instruction = row['instruction']
    input_value = row['input']
    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}
    ]
    
    response = predict(messages, model, tokenizer)
    match = re.search(pattern, response)
    if match:
        # 提取匹配的表名字符串
        table_names_str = match.group(1)

        # 使用中文逗号分隔表名，这里我们替换中文逗号为英文逗号并去除空格
#         table_names = [name.strip() for name in table_names_str.replace('，', ',').split(',')]
        table_names = table_names_str.split(', ')

        # 将table_names整合成一个由英文逗号分隔的字符串
        predict_table = ','.join(table_names)
        print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
        print("预测表名：", predict_table)
        print("############################################")
    else:
        predict_table = "None"
    predict_list.append(predict_table)
#     messages.append({"role": "assistant", "content": f"{response}"})
#     result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
#     print(result_text)
    
#     test_text_list.append(swanlab.Text(result_text, caption=response))

```Reference Table
-- Tables: singer;
```
++++++++++++++++index:1++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:2++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:3++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:4++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:5++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:6++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:7++++++++++++++++++++++++
预测表名： singer
#########

```Reference Table
-- Tables: student, has_pet, pets;
```
++++++++++++++++index:53++++++++++++++++++++++++
预测表名： student,has_pet,pets
############################################
```Reference Table
-- Tables: student, has_pet, pets;
```
++++++++++++++++index:54++++++++++++++++++++++++
预测表名： student,has_pet,pets
############################################
```Reference Table
-- Tables: student, has_pet, pets;
```
++++++++++++++++index:55++++++++++++++++++++++++
预测表名： student,has_pet,pets
############################################
```Reference Table
-- Tables: pets;
```
++++++++++++++++index:56++++++++++++++++++++++++
预测表名： pets
############################################
```Reference Table
-- Tables: pets;
```
++++++++++++++++index:57++++++++++++++++++++++++
预测表名： pets
############################################
```Reference Table
-- Tables: student, has_pet, pets;
```
++++++++++++++++index:58++++++++++++++++++++++++
预测表名： student,has_pet,pets
#######################################

```Reference Table
-- Tables: car_makers, car_names, cars_data;
```
++++++++++++++++index:103++++++++++++++++++++++++
预测表名： car_makers,car_names,cars_data
############################################
```Reference Table
-- Tables: model_list, car_names, cars_data;
```
++++++++++++++++index:104++++++++++++++++++++++++
预测表名： model_list,car_names,cars_data
############################################
```Reference Table
-- Tables: car_names, cars_data;
```
++++++++++++++++index:105++++++++++++++++++++++++
预测表名： car_names,cars_data
############################################
```Reference Table
-- Tables: continents, countries, car_makers;
```
++++++++++++++++index:106++++++++++++++++++++++++
预测表名： continents,countries,car_makers
############################################
```Reference Table
-- Tables: continents, car_makers;
```
++++++++++++++++index:107++++++++++++++++++++++++
预测表名： continents,car_makers
############################################
```Reference Table
-- Tables: car_makers

```Reference Table
-- Tables: car_makers, model_list;
```
++++++++++++++++index:151++++++++++++++++++++++++
预测表名： car_makers,model_list
############################################
```Reference Table
-- Tables: car_makers, model_list, car_names, cars_data;
```
++++++++++++++++index:152++++++++++++++++++++++++
预测表名： car_makers,model_list,car_names,cars_data
############################################
```Reference Table
-- Tables: car_makers, model_list, car_names, cars_data;
```
++++++++++++++++index:153++++++++++++++++++++++++
预测表名： car_makers,model_list,car_names,cars_data
############################################
```Reference Table
-- Tables: cars_data;
```
++++++++++++++++index:154++++++++++++++++++++++++
预测表名： cars_data
############################################
```Reference Table
-- Tables: cars_data;
```
++++++++++++++++index:155++++++++++++++++++++++++
预测表名： cars_data
############################################
```Reference Table
-- Tables: cars_data;
```
++++++++++++++++

```Reference Table
-- Tables: airports;
```
++++++++++++++++index:200++++++++++++++++++++++++
预测表名： airports
############################################
```Reference Table
-- Tables: airports;
```
++++++++++++++++index:201++++++++++++++++++++++++
预测表名： airports
############################################
```Reference Table
-- Tables: airports;
```
++++++++++++++++index:202++++++++++++++++++++++++
预测表名： airports
############################################
```Reference Table
-- Tables: airports;
```
++++++++++++++++index:203++++++++++++++++++++++++
预测表名： airports
############################################
```Reference Table
-- Tables: airports, flights;
```
++++++++++++++++index:204++++++++++++++++++++++++
预测表名： airports,flights
############################################
```Reference Table
-- Tables: airports, flights;
```
++++++++++++++++index:205++++++++++++++++++++++++
预测表名： airports,flights
############################################
```Reference Table
-- Tables: airports, fl

...

```Reference Table
-- Tables: country;
```
++++++++++++++++index:804++++++++++++++++++++++++
预测表名： country
############################################
```Reference Table
-- Tables: country;
```
++++++++++++++++index:805++++++++++++++++++++++++
预测表名： country
############################################
```Reference Table
-- Tables: country;
```
++++++++++++++++index:806++++++++++++++++++++++++
预测表名： country
############################################
```Reference Table
-- Tables: country;
```
++++++++++++++++index:807++++++++++++++++++++++++
预测表名： country
############################################
```Reference Table
-- Tables: country;
```
++++++++++++++++index:808++++++++++++++++++++++++
预测表名： country
############################################
```Reference Table
-- Tables: country;
```
++++++++++++++++index:809++++++++++++++++++++++++
预测表名： country
############################################
```Reference Table
-- Tables: country;
```
++++++++++++++++index:810++++++++++++++++++++

```Reference Table
-- Tables: orchestra, performance;
```
++++++++++++++++index:856++++++++++++++++++++++++
预测表名： orchestra,performance
############################################
```Reference Table
-- Tables: orchestra;
```
++++++++++++++++index:857++++++++++++++++++++++++
预测表名： orchestra
############################################
```Reference Table
-- Tables: orchestra;
```
++++++++++++++++index:858++++++++++++++++++++++++
预测表名： orchestra
############################################
```Reference Table
-- Tables: orchestra;
```
++++++++++++++++index:859++++++++++++++++++++++++
预测表名： orchestra
############################################
```Reference Table
-- Tables: orchestra;
```
++++++++++++++++index:860++++++++++++++++++++++++
预测表名： orchestra
############################################
```Reference Table
-- Tables: orchestra, performance;
```
++++++++++++++++index:861++++++++++++++++++++++++
预测表名： orchestra,performance
############################################
```Reference T

```Reference Table
-- Tables: highschooler, likes;
```
++++++++++++++++index:906++++++++++++++++++++++++
预测表名： highschooler,likes
############################################
```Reference Table
-- Tables: highschooler, likes;
```
++++++++++++++++index:907++++++++++++++++++++++++
预测表名： highschooler,likes
############################################
```Reference Table
-- Tables: highschooler, likes;
```
++++++++++++++++index:908++++++++++++++++++++++++
预测表名： highschooler,likes
############################################
```Reference Table
-- Tables: highschooler, likes;
```
++++++++++++++++index:909++++++++++++++++++++++++
预测表名： highschooler,likes
############################################
```Reference Table
-- Tables: highschooler, likes;
```
++++++++++++++++index:910++++++++++++++++++++++++
预测表名： highschooler,likes
############################################
```Reference Table
-- Tables: highschooler, friend;
```
++++++++++++++++index:911++++++++++++++++++++++++
预测表名： highschooler,

```Reference Table
-- Tables: owners, dogs, sizes;
```
++++++++++++++++index:952++++++++++++++++++++++++
预测表名： owners,dogs,sizes
############################################
```Reference Table
-- Tables: owners, dogs;
```
++++++++++++++++index:953++++++++++++++++++++++++
预测表名： owners,dogs
############################################
```Reference Table
-- Tables: owners, dogs;
```
++++++++++++++++index:954++++++++++++++++++++++++
预测表名： owners,dogs
############################################
```Reference Table
-- Tables: breeds, dogs;
```
++++++++++++++++index:955++++++++++++++++++++++++
预测表名： breeds,dogs
############################################
```Reference Table
-- Tables: breeds, dogs, treatments;
```
++++++++++++++++index:956++++++++++++++++++++++++
预测表名： breeds,dogs,treatments
############################################
```Reference Table
-- Tables: owners, dogs;
```
++++++++++++++++index:957++++++++++++++++++++++++
预测表名： owners,dogs
###########################################

```Reference Table
-- Tables: singer;
```
++++++++++++++++index:1003++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:1004++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:1005++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:1006++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:1007++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:1008++++++++++++++++++++++++
预测表名： singer
############################################
```Reference Table
-- Tables: singer;
```
++++++++++++++++index:1009++++++++++++++++++++++++
预